<a href="https://colab.research.google.com/github/nikgilae/Camera/blob/main/%D0%B4%D0%BE%D0%BE%D0%B1%D1%83%D1%87%D0%B5%D0%BD%D0%B8%D0%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install transformers datasets accelerate --upgrade -q
#!pip install scikit-learn pandas -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 18.0 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, recall_score, classification_report
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)
from datasets import Dataset
import os

print("="*70)
print("🔧 БЕЗОПАСНОЕ ДООБУЧЕНИЕ МОДЕЛИ (с проверками)")
print("="*70)

🔧 БЕЗОПАСНОЕ ДООБУЧЕНИЕ МОДЕЛИ (с проверками)


In [ ]:
CSV_PATH = "/content/drive/MyDrive/s7_airlines_reviews_with_sentiment.csv"
OUTPUT_DIR = "/content/drive/MyDrive/finetuned_sentiment_model_v2"  # Новая версия!
BASE_MODEL_NAME = "tabularisai/multilingual-sentiment-analysis"

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"\n🖥️  Устройство: {device.upper()}")


🖥️  Устройство: CUDA


In [ ]:
print("\n" + "="*70)
print("1️⃣ ПРОВЕРКА БАЗОВОЙ МОДЕЛИ")
print("="*70)

base_model = AutoModelForSequenceClassification.from_pretrained(BASE_MODEL_NAME)
base_config = base_model.config

print(f"\n📋 Конфигурация базовой модели:")
print(f"   Количество классов: {base_config.num_labels}")
print(f"   ID → Label: {base_config.id2label}")
print(f"   Label → ID: {base_config.label2id}")

# КРИТИЧЕСКИ ВАЖНО: Запомнить правильный маппинг
CORRECT_LABEL_MAPPING = base_config.label2id
print(f"\n✅ Используем маппинг: {CORRECT_LABEL_MAPPING}")


1️⃣ ПРОВЕРКА БАЗОВОЙ МОДЕЛИ


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/851 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/541M [00:00<?, ?B/s]


📋 Конфигурация базовой модели:
   Количество классов: 5
   ID → Label: {0: 'Very Negative', 1: 'Negative', 2: 'Neutral', 3: 'Positive', 4: 'Very Positive'}
   Label → ID: {'Negative': 1, 'Neutral': 2, 'Positive': 3, 'Very Negative': 0, 'Very Positive': 4}

✅ Используем маппинг: {'Negative': 1, 'Neutral': 2, 'Positive': 3, 'Very Negative': 0, 'Very Positive': 4}


In [ ]:
print("\n" + "="*70)
print("2️⃣ ЗАГРУЗКА И ПРОВЕРКА ДАННЫХ")
print("="*70)

df = pd.read_csv(CSV_PATH)
print(f"\n✅ Загружено строк: {len(df)}")
print(f"📋 Колонки: {df.columns.tolist()}")

# Проверка на пропуски
if df.isnull().sum().sum() > 0:
    print(f"\n⚠️  Обнаружены пропуски:")
    print(df.isnull().sum())
    print("   Удаляем строки с пропусками...")
    df = df.dropna()
    print(f"   Осталось строк: {len(df)}")

# Проверка колонок
if 'Review' in df.columns and 'Sentiment' in df.columns:
    df = df.rename(columns={'Review': 'text', 'Sentiment': 'label'})
elif 'text' not in df.columns or 'label' not in df.columns:
    print(f"\n❌ ОШИБКА: Не найдены нужные колонки!")
    print(f"   Найдены: {df.columns.tolist()}")
    print(f"   Нужны: 'text' и 'label' (или 'Review' и 'Sentiment')")
    raise ValueError("Неправильная структура данных")

print(f"\n📊 Распределение классов:")
print(df['label'].value_counts())

# КРИТИЧЕСКАЯ ПРОВЕРКА: метки в данных
unique_labels = df['label'].unique()
print(f"\n🔍 Уникальные метки в данных: {unique_labels}")

# Нормализация меток к нижнему регистру
df['label'] = df['label'].str.lower()

# Проверка что все метки известны
valid_labels = set(['positive', 'negative', 'neutral'])
data_labels = set(df['label'].unique())

if not data_labels.issubset(valid_labels):
    unknown = data_labels - valid_labels
    print(f"\n❌ ОШИБКА: Неизвестные метки в данных: {unknown}")
    print(f"   Допустимые метки: {valid_labels}")
    raise ValueError("Неправильные метки в данных")

print(f"\n✅ Все метки корректны: {data_labels}")

# Преобразование строковых меток в числа (ИСПОЛЬЗУЯ ПРАВИЛЬНЫЙ МАППИНГ!)
print(f"\n🔄 Преобразование меток в числа...")

# Создаём обратный маппинг из конфига базовой модели
id2label = base_config.id2label
label2id = {v.lower(): k for k, v in id2label.items()}

print(f"   Используем маппинг: {label2id}")

df['label_int'] = df['label'].map(label2id)

# Проверка что все метки сопоставились
if df['label_int'].isnull().any():
    print(f"\n❌ ОШИБКА: Не все метки удалось преобразовать!")
    print(df[df['label_int'].isnull()]['label'].value_counts())
    raise ValueError("Ошибка преобразования меток")

print(f"\n✅ Метки преобразованы:")
print(df[['label', 'label_int']].drop_duplicates().sort_values('label_int'))


2️⃣ ЗАГРУЗКА И ПРОВЕРКА ДАННЫХ

✅ Загружено строк: 8740
📋 Колонки: ['Review', 'Sentiment']

📊 Распределение классов:
label
positive    5175
negative    2760
neutral      805
Name: count, dtype: int64

🔍 Уникальные метки в данных: ['positive' 'neutral' 'negative']

✅ Все метки корректны: {'negative', 'positive', 'neutral'}

🔄 Преобразование меток в числа...
   Используем маппинг: {'very negative': 0, 'negative': 1, 'neutral': 2, 'positive': 3, 'very positive': 4}

✅ Метки преобразованы:
       label  label_int
12  negative          1
10   neutral          2
0   positive          3


In [ ]:
print("\n" + "="*70)
print("3️⃣ РАЗДЕЛЕНИЕ НА TRAIN/TEST")
print("="*70)

train_df, test_df = train_test_split(
    df[['text', 'label_int']],
    test_size=0.3,
    random_state=42,
    stratify=df['label_int']
)

print(f"\n📊 Train: {len(train_df)} строк")
print(f"📊 Test:  {len(test_df)} строк")

print(f"\n📊 Распределение в Train:")
print(train_df['label_int'].value_counts().sort_index())
print(f"\n📊 Распределение в Test:")
print(test_df['label_int'].value_counts().sort_index())

# Проверка баланса классов
class_counts = train_df['label_int'].value_counts()
imbalance_ratio = class_counts.max() / class_counts.min()

if imbalance_ratio > 3:
    print(f"\n⚠️  ВНИМАНИЕ: Дисбаланс классов = {imbalance_ratio:.1f}:1")
    print("   Это может ухудшить качество модели!")
else:
    print(f"\n✅ Баланс классов приемлемый ({imbalance_ratio:.1f}:1)")


3️⃣ РАЗДЕЛЕНИЕ НА TRAIN/TEST

📊 Train: 6118 строк
📊 Test:  2622 строк

📊 Распределение в Train:
label_int
1    1932
2     564
3    3622
Name: count, dtype: int64

📊 Распределение в Test:
label_int
1     828
2     241
3    1553
Name: count, dtype: int64

⚠️  ВНИМАНИЕ: Дисбаланс классов = 6.4:1
   Это может ухудшить качество модели!


In [ ]:
train_dataset = Dataset.from_pandas(train_df.reset_index(drop=True))
test_dataset = Dataset.from_pandas(test_df.reset_index(drop=True))

# Переименование для Trainer
train_dataset = train_dataset.rename_column('label_int', 'label')
test_dataset = test_dataset.rename_column('label_int', 'label')

In [ ]:
print("\n" + "="*70)
print("4️⃣ ТОКЕНИЗАЦИЯ")
print("="*70)

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_NAME)

def tokenize_function(examples):
    return tokenizer(
        examples['text'],
        padding=False,
        truncation=True,
        max_length=512
    )

print("\n🔄 Токенизация данных...")
tokenized_train = train_dataset.map(tokenize_function, batched=True, remove_columns=['text'])
tokenized_test = test_dataset.map(tokenize_function, batched=True, remove_columns=['text'])
print("✅ Токенизация завершена!")

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


4️⃣ ТОКЕНИЗАЦИЯ


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]


🔄 Токенизация данных...


Map:   0%|          | 0/6118 [00:00<?, ? examples/s]

Map:   0%|          | 0/2622 [00:00<?, ? examples/s]

✅ Токенизация завершена!


In [ ]:
print("\n" + "="*70)
print("5️⃣ ЗАГРУЗКА МОДЕЛИ")
print("="*70)

# ВАЖНО: Загружаем заново, чтобы гарантировать чистое состояние
model = AutoModelForSequenceClassification.from_pretrained(
    BASE_MODEL_NAME,
    num_labels=base_config.num_labels,
    id2label=base_config.id2label,
    label2id=base_config.label2id
)

print(f"\n✅ Модель загружена с правильной конфигурацией:")
print(f"   ID → Label: {model.config.id2label}")
print(f"   Label → ID: {model.config.label2id}")


5️⃣ ЗАГРУЗКА МОДЕЛИ

✅ Модель загружена с правильной конфигурацией:
   ID → Label: {0: 'Very Negative', 1: 'Negative', 2: 'Neutral', 3: 'Positive', 4: 'Very Positive'}
   Label → ID: {'Negative': 1, 'Neutral': 2, 'Positive': 3, 'Very Negative': 0, 'Very Positive': 4}


In [ ]:
print("\n" + "="*70)
print("6️⃣ ОЦЕНКА БАЗОВОЙ МОДЕЛИ (ДО ДООБУЧЕНИЯ)")
print("="*70)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    accuracy = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='weighted')
    recall = recall_score(labels, predictions, average='weighted')

    return {
        'accuracy': accuracy,
        'f1_weighted': f1,
        'recall_weighted': recall
    }

# Создаём временный TrainingArguments для оценки, чтобы избежать запроса wandb
temp_training_args = TrainingArguments(
    output_dir="./temp_output", # Временная директория, не будет использоваться
    report_to="none",           # Отключить логирование в wandb
    per_device_eval_batch_size=32,
    seed=42
)

# Создаём временный Trainer только для оценки
temp_trainer = Trainer(
    model=model,
    args=temp_training_args,    # Передаем временные аргументы
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

baseline_metrics = temp_trainer.evaluate(tokenized_test)
print(f"\n📊 Baseline (до обучения):")
print(f"   Accuracy: {baseline_metrics['eval_accuracy']:.2%}")
print(f"   F1:       {baseline_metrics['eval_f1_weighted']:.4f}")
print(f"   Recall:   {baseline_metrics['eval_recall_weighted']:.4f}")

# КРИТИЧЕСКАЯ ПРОВЕРКА: baseline не должен быть слишком плохим
if baseline_metrics['eval_accuracy'] < 0.3:
    print(f"\n⚠️  ВНИМАНИЕ: Baseline accuracy слишком низкая ({baseline_metrics['eval_accuracy']:.2%})")
    print("   Возможно проблема с маппингом меток!")
    print("   Рекомендуется проверить данные перед продолжением.")

    response = input("\nПродолжить обучение? (yes/no): ")
    if response.lower() != 'yes':
        print("❌ Обучение отменено.")
        raise SystemExit()


6️⃣ ОЦЕНКА БАЗОВОЙ МОДЕЛИ (ДО ДООБУЧЕНИЯ)



📊 Baseline (до обучения):
   Accuracy: 38.98%
   F1:       0.4895
   Recall:   0.3898


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
print("\n" + "="*70)
print("7️⃣ НАСТРОЙКА ПАРАМЕТРОВ ОБУЧЕНИЯ")
print("="*70)

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,

    # КОНСЕРВАТИВНЫЕ ПАРАМЕТРЫ (защита от переобучения)
    num_train_epochs=2,              # Меньше эпох
    learning_rate=5e-6,              # В 4 РАЗА меньше! (было 2e-5)

    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,

    # Больше регуляризации
    weight_decay=0.02,               # Больше weight decay (было 0.01)
    warmup_steps=500,

    # Частая оценка для раннего обнаружения проблем
    eval_strategy="steps",           # Оценка каждые N шагов
    eval_steps=100,                  # Оценка каждые 100 шагов
    save_strategy="steps",
    save_steps=100,

    logging_steps=50,

    load_best_model_at_end=True,
    metric_for_best_model="f1_weighted",
    greater_is_better=True,

    # Оптимизация
    fp16=torch.cuda.is_available(),
    gradient_accumulation_steps=2,

    seed=42,
    push_to_hub=False,
    report_to="none"
)

print("\n✅ Параметры обучения (консервативные):")
print(f"   Эпох: {training_args.num_train_epochs}")
print(f"   Learning rate: {training_args.learning_rate}")
print(f"   Weight decay: {training_args.weight_decay}")
print(f"   Оценка каждые {training_args.eval_steps} шагов")



7️⃣ НАСТРОЙКА ПАРАМЕТРОВ ОБУЧЕНИЯ

✅ Параметры обучения (консервативные):
   Эпох: 2
   Learning rate: 5e-06
   Weight decay: 0.02
   Оценка каждые 100 шагов


In [ ]:
print("\n" + "="*70)
print("8️⃣ НАЧАЛО ОБУЧЕНИЯ")
print("="*70)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

print("\n🚀 Запуск обучения...")
print("   Следи за метриками! Если accuracy падает - останови процесс (Runtime → Interrupt)")
print()

train_result = trainer.train()

print("\n✅ Обучение завершено!")



8️⃣ НАЧАЛО ОБУЧЕНИЯ

🚀 Запуск обучения...
   Следи за метриками! Если accuracy падает - останови процесс (Runtime → Interrupt)



/tmp/ipython-input-1835152884.py:5: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss,Validation Loss,Accuracy,F1 Weighted,Recall Weighted
100,0.695100,0.384657,0.895881,0.886800,0.895881
200,0.089400,0.047928,0.984744,0.984234,0.984744
300,0.017300,0.010084,1.000000,1.000000,1.000000


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


KeyboardInterrupt: 

In [ ]:
print("\n" + "="*70)
print("9️⃣ ФИНАЛЬНАЯ ОЦЕНКА")
print("="*70)

final_metrics = trainer.evaluate()

print(f"\n📊 РЕЗУЛЬТАТЫ:")
print(f"\n   BASELINE (до обучения):")
print(f"      Accuracy: {baseline_metrics['eval_accuracy']:.2%}")
print(f"      F1:       {baseline_metrics['eval_f1_weighted']:.4f}")
print(f"\n   ПОСЛЕ ДООБУЧЕНИЯ:")
print(f"      Accuracy: {final_metrics['eval_accuracy']:.2%}")
print(f"      F1:       {final_metrics['eval_f1_weighted']:.4f}")
print(f"\n   ИЗМЕНЕНИЕ:")
print(f"      Accuracy: {(final_metrics['eval_accuracy'] - baseline_metrics['eval_accuracy'])*100:+.1f}%")
print(f"      F1:       {(final_metrics['eval_f1_weighted'] - baseline_metrics['eval_f1_weighted']):+.4f}")

# КРИТИЧЕСКАЯ ПРОВЕРКА: модель не должна ухудшиться
if final_metrics['eval_accuracy'] < baseline_metrics['eval_accuracy'] - 0.05:
    print(f"\n❌ ПРОБЛЕМА: Модель УХУДШИЛАСЬ на {(baseline_metrics['eval_accuracy'] - final_metrics['eval_accuracy'])*100:.1f}%!")
    print("   НЕ СОХРАНЯЕМ эту модель!")
    print("\n💡 Возможные причины:")
    print("   - Слишком большой learning rate (попробуй ещё меньше)")
    print("   - Слишком много эпох")
    print("   - Плохое качество данных")
    print("   - Неправильный маппинг меток")

    save_model = input("\nВсё равно сохранить модель? (yes/no): ")
    if save_model.lower() != 'yes':
        print("❌ Модель НЕ сохранена.")
        raise SystemExit()

# Детальный отчёт
predictions = trainer.predict(tokenized_test)
pred_labels = np.argmax(predictions.predictions, axis=1)
true_labels = predictions.label_ids

print("\n" + "="*70)
print("📋 ДЕТАЛЬНЫЙ ОТЧЁТ")
print("="*70)
print()
print(classification_report(
    true_labels,
    pred_labels,
    target_names=[id2label[i] for i in sorted(id2label.keys())],
    digits=4
))


9️⃣ ФИНАЛЬНАЯ ОЦЕНКА


Step,Training Loss,Validation Loss,Accuracy,F1 Weighted,Recall Weighted
100,0.695100,0.384657,0.895881,0.886800,0.895881
200,0.089400,0.047928,0.984744,0.984234,0.984744
300,0.017300,0.010084,1.000000,1.000000,1.000000
346,0.017300,0.007235,1.000000,1.000000,1.000000



📊 РЕЗУЛЬТАТЫ:

   BASELINE (до обучения):
      Accuracy: 38.98%
      F1:       0.4895

   ПОСЛЕ ДООБУЧЕНИЯ:
      Accuracy: 100.00%
      F1:       1.0000

   ИЗМЕНЕНИЕ:
      Accuracy: +61.0%
      F1:       +0.5105

📋 ДЕТАЛЬНЫЙ ОТЧЁТ



ValueError: Number of classes, 3, does not match size of target_names, 5. Try specifying the labels parameter

In [ ]:
print("\n" + "="*50)
print("СОХРАНЕНИЕ МОДЕЛИ")
print("="*50 + "\n")

# Основная папка для модели
final_model_path = "/content/drive/MyDrive/finetuned_sentiment_model_final"

# Сохранение модели и токенизатора
trainer.save_model(final_model_path)
tokenizer.save_pretrained(final_model_path)

print(f"✅ Модель сохранена в: {final_model_path}")
print(f"✅ Токенизатор сохранён в: {final_model_path}")

# Сохранение метрик
metrics_path = os.path.join(final_model_path, "metrics.txt")
with open(metrics_path, 'w', encoding='utf-8') as f:
    f.write("МЕТРИКИ ДООБУЧЕННОЙ МОДЕЛИ\n")
    f.write("="*50 + "\n\n")
    f.write(f"Accuracy: {final_metrics['eval_accuracy']:.4f}\n")
    f.write(f"F1 (macro): {final_metrics['eval_f1_macro']:.4f}\n")
    f.write(f"F1 (weighted): {final_metrics['eval_f1_weighted']:.4f}\n")
    f.write(f"Recall (macro): {final_metrics['eval_recall_macro']:.4f}\n")
    f.write(f"Recall (weighted): {final_metrics['eval_recall_weighted']:.4f}\n\n")
    f.write("ДЕТАЛЬНЫЙ ОТЧЁТ:\n")
    f.write(classification_report(true_labels, pred_labels, target_names=target_names))

print(f"✅ Метрики сохранены в: {metrics_path}")

print("\n" + "="*50)
print("ВСЁ ГОТОВО!")
print("="*50)
print(f"\nТвоя дообученная модель находится здесь:")
print(f"📁 {final_model_path}")
print(f"\nФайлы в папке:")
print(f"  - pytorch_model.bin (или model.safetensors)")
print(f"  - config.json")
print(f"  - tokenizer_config.json")
print(f"  - vocab.txt (или другие файлы токенизатора)")
print(f"  - metrics.txt")


СОХРАНЕНИЕ МОДЕЛИ

✅ Модель сохранена в: /content/drive/MyDrive/finetuned_sentiment_model_final
✅ Токенизатор сохранён в: /content/drive/MyDrive/finetuned_sentiment_model_final


KeyError: 'eval_f1_macro'

In [ ]:
if final_metrics['eval_accuracy'] >= baseline_metrics['eval_accuracy'] - 0.05:
    print("\n" + "="*70)
    print("💾 СОХРАНЕНИЕ МОДЕЛИ")
    print("="*70)

    final_model_path = OUTPUT_DIR + "_final"
    trainer.save_model(final_model_path)
    tokenizer.save_pretrained(final_model_path)

    # Сохранение метрик
    with open(f"{final_model_path}/metrics.txt", 'w') as f:
        f.write(f"BASELINE:\n")
        f.write(f"  Accuracy: {baseline_metrics['eval_accuracy']:.4f}\n")
        f.write(f"  F1: {baseline_metrics['eval_f1_weighted']:.4f}\n\n")
        f.write(f"AFTER FINETUNING:\n")
        f.write(f"  Accuracy: {final_metrics['eval_accuracy']:.4f}\n")
        f.write(f"  F1: {final_metrics['eval_f1_weighted']:.4f}\n\n")
        f.write(f"IMPROVEMENT:\n")
        f.write(f"  Accuracy: {(final_metrics['eval_accuracy'] - baseline_metrics['eval_accuracy'])*100:+.2f}%\n")
        f.write(f"  F1: {(final_metrics['eval_f1_weighted'] - baseline_metrics['eval_f1_weighted']):+.4f}\n")

    print(f"\n✅ Модель сохранена: {final_model_path}")
    print(f"✅ Метрики сохранены: {final_model_path}/metrics.txt")

    print("\n" + "="*70)
    print("✅ ВСЁ ГОТОВО!")
    print("="*70)
    print(f"\nМодель {'УЛУЧШИЛАСЬ' if final_metrics['eval_accuracy'] > baseline_metrics['eval_accuracy'] else 'НЕ УХУДШИЛАСЬ'} после дообучения")
    print(f"Можно использовать в production!")
else:
    print("\n❌ Модель не сохранена из-за ухудшения качества.")


💾 СОХРАНЕНИЕ МОДЕЛИ

✅ Модель сохранена: /content/drive/MyDrive/finetuned_sentiment_model_v2_final
✅ Метрики сохранены: /content/drive/MyDrive/finetuned_sentiment_model_v2_final/metrics.txt

✅ ВСЁ ГОТОВО!

Модель УЛУЧШИЛАСЬ после дообучения
Можно использовать в production!
